# Library QC with long-read sequencing

### Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import igv_notebook
from report_utils import *

igv_notebook.init()

sns.set_theme()
sns.set_style('darkgrid')
pioneer_colors = ['#FF8633', '#423759', '#314942', '#FFA632', '#F7F3ED']
sns.set_palette(sns.color_palette(pioneer_colors))

result_dir = "."


### Get sample info

In [ ]:
samples = pd.read_csv("samples.csv")
samps = {x:os.path.join(result_dir, x) for x in samples.id.to_list()}
print(f'Analyzing samples: {", ".join(samps)}')

### Load all the files

In [3]:
# barcodes
bc_seqs = list_files(samps, "barcodes.tsv")
bc_counts = list_files(samps, "barcode_counts.tsv")
bc_files = {}
for key, val in bc_seqs.items():
    bc_files[key] = [val, bc_counts[key]]

barcode_data = pd.concat([load_barcode_data(val, key) for key, val in list_files(samps, "barcodes.tsv").items()])
barcode_counts = pd.concat([load_barcode_counts(val, key) for key, val in list_files(samps, "barcode_counts.tsv").items()])

# Insert
insert_data = pd.concat([load_insert_data(val, key) for key, val in list_files(samps, "inserts.tsv").items()])

# genome coverage
genome_cov_data = pd.concat([load_genome_cov(val, key) for key, val in list_files(samps, "genome_coverage.tsv").items()])

# gene coverage
gene_cov_data = pd.concat([load_gene_cov(val, key) for key, val in list_files(samps, "gene_coverage.bed").items()])

# insert coverage
insert_cov_data = pd.concat([load_insert_cov(val, key) for key, val in list_files(samps, "insert_coverage.bed").items()])
insert_cov_full = pd.concat([load_insert_cov(val, key) for key, val in list_files(samps, "insert_coverage_full.bed").items()])
intersect = pd.concat([load_intersects(val, key) for key, val in list_files(samps, "insert_intersect.out").items()])

# sequence stats
seq_stat = pd.concat([load_seq_stats(val, key) for key, val in list_files(samps, "seq_stats.tsv").items()])

# coverage stats
cov_stat = pd.concat([load_cov_stats(val, key) for key, val in list_files(samps, "genome_cov_stats.tsv").items()])

# barcode and insert detection stats
barcode_detect_stats = load_cutadapt_report(samps, "cutadapt_barcode_report.json")
insert_detect_stats = load_cutadapt_report(samps, "cutadapt_inserts_report.json")


# Summarize sequence info
num_seqs = seq_summary(barcode_data, insert_data, seq_stat)

#### Write them out for easy access in the future

In [4]:
barcode_data.to_csv(os.path.join(result_dir, 'barcode_data.tsv'))
barcode_counts.to_csv(os.path.join(result_dir, 'barcode_counts.tsv'))
insert_data.to_csv(os.path.join(result_dir, 'insert_data.tsv'))
genome_cov_data.to_csv(os.path.join(result_dir, 'genome_cov_data.tsv'))
gene_cov_data.to_csv(os.path.join(result_dir, 'gene_cov_data.tsv'))
insert_cov_data.to_csv(os.path.join(result_dir, 'insert_cov_data.tsv'))
intersect.to_csv(os.path.join(result_dir, 'intersect_data.csv'))
num_seqs.to_csv(os.path.join(result_dir, 'seq_stats.csv'))


## Summary

### Read Stats

In [ ]:
num_seqs

### Insert size distribution

In [ ]:
gr = sns.FacetGrid(insert_data, col = 'sample', sharey = False, height=5, col_wrap = 5)
gr.map(sns.histplot, 'insert_len')
gr.set_titles(col_template = '{col_name}')
gr.set_axis_labels('Insert Length', 'Number of Inserts')


## Coverage Analysis 

#### Coverage stats

In [ ]:
cov_stat.style.format(precision = 3, thousands = ",").format_index(str.title, axis = 1)

### Genome coverage

In [ ]:
gr = sns.FacetGrid(genome_cov_data, col = 'sample', height = 4, col_wrap = 5)
gr.map(sns.lineplot, "pos", "cov")
gr.set_titles(col_template = '{col_name}')
gr.set_axis_labels('Genomic position', 'Read depth')


In [ ]:
sns.barplot(cov_stat, x = "sample", y = "coverage")
plt.xlabel("Sample")
plt.ylabel("% of genome covered")

### Genes per insert

#### Percentage of inserts that have X full genes

In [ ]:
gr = sns.displot(insert_cov_full, x = "count", col = "sample", discrete = True,
            stat="percent", common_norm = False)
gr.set_titles(col_template = '{col_name}')
gr.set_axis_labels('Number of full genes per insert', '% of inserts')

#### Percentage of inserts that have X genes overlapping by any amount

In [ ]:
gr = sns.displot(insert_cov_data, x = "count", col = "sample", discrete = True,
            stat="percent", common_norm = False)
gr.set_titles(col_template = '{col_name}')
gr.set_axis_labels('Number of genes per insert', '% of inserts')

### Genes covered by insert

The xaxis is the count of the number of inserts per gene and the yaxis is the number of genes that have that many inserts overlapping.

In [ ]:
gr = sns.displot(gene_cov_data, x = "count", col = "sample", discrete = True,
            stat="percent", common_norm = False)
gr.set_titles(col_template = '{col_name}')
gr.set_axis_labels('Number of overlapping inserts', '% of genes')

#### Percentage of gene overlapping an insert

Percent of genes in each sample overlapping at least 1 insert

In [ ]:
gene_cov_summary = gene_cov_data[gene_cov_data.percent_cov > 0].groupby('sample', as_index=False).agg("size")
n_genes = gene_cov_data.groupby('sample', as_index = False).agg('size').rename(columns = {'size': 'n_genes'})
gene_cov_summary = gene_cov_summary.merge(n_genes, on = 'sample')
gene_cov_summary['pct_non_zero'] = 100 * (gene_cov_summary['size'] / gene_cov_summary['n_genes'])
gene_cov_summary

Percent of genes in each sample completely covered by inserts

In [ ]:
gene_cov_summary = gene_cov_data[gene_cov_data.percent_cov == 1].groupby('sample', as_index=False).agg("size")
n_genes = gene_cov_data.groupby('sample', as_index = False).agg('size').rename(columns = {'size': 'n_genes'})
gene_cov_summary = gene_cov_summary.merge(n_genes, on = 'sample')
gene_cov_summary['pct_100_cov'] = 100 * (gene_cov_summary['size'] / gene_cov_summary['n_genes'])
gene_cov_summary

In [ ]:
gr = sns.displot(gene_cov_data, x = "percent_cov", col = "sample", stat="percent", common_norm = False)
gr.set_titles(col_template = '{col_name}')
gr.set_axis_labels('Fraction of gene covered by inserts', '% of genes')

#### Percent of insert covered by genes

In [ ]:
gr = sns.displot(insert_cov_data, x = "percent_cov", col = 'sample', stat="percent", common_norm = False)
gr.set_titles(col_template = '{col_name}')
gr.set_axis_labels('Fraction of insert covered', "% of inserts")


### Percentage of genes covered by at least X inserts

In [19]:
gene_agg = gene_cov_data.groupby(['sample', 'count'], as_index=False).agg("size")
gene_agg['pct'] = gene_agg.groupby('sample')['size'].apply(lambda x: 100*(x / x.sum())).reset_index(drop = True)
gene_count_sorted = gene_agg.sort_values(['sample', 'count'], ascending = False)
grouped = gene_count_sorted.groupby('sample')['pct']
gene_count_sorted['cummulative_pct'] = grouped.cumsum()

In [ ]:
gr = sns.FacetGrid(gene_count_sorted, col = 'sample', col_order = list(samps.keys()), height = 4.5)
gr.map(sns.lineplot, "count", "cummulative_pct")
gr.set_titles(col_template = '{col_name}')
gr.set_axis_labels('Number of inserts', '% of genes')

### View insert alignments

In [21]:
sample = ''
fasta_path = os.path.realpath(os.path.expanduser("~/shared/genomes/H_elongata/H_elongata_contigs.fna"))
fai_path = os.path.realpath(os.path.expanduser("~/shared/genomes/H_elongata/H_elongata_contigs.fna.fai"))
anno_path = os.path.realpath(os.path.expanduser("~/shared/genomes/H_elongata/H_elongata_annotations.gff"))

In [ ]:
igv_browser = igv_notebook.Browser(
    {
        "reference": {
            "id": "Helongata",
            "name": "Helongata",
            "fastaPath": fasta_path,
            "indexPath": fai_path,
            "tracks": [
                {
                    "name": "genes",
                    "path": anno_path,
                    "height": 100,
                    "filterTypes": ['region', 'CDS']
                }
            ]
        }
    }
)
igv_browser.load_track(
    {
                    "name": "inserts",
                    "path": os.path.join(sample, "mapped_inserts.bam"),
                    "visibilityWindow": 4300000,
                    "showAlignments": True,
                    "showCoverage": False,
                    "format": "bam",
                    "type": "alignment",
                    "height": 100,
                    "CoverageColour": "black",
                    "coverageTrackHeight": 70
                }
)